---
title: Transpile circuits remotely with the Qiskit Transpiler Service
description: What is the Qiskit Transpiler Service and how to use it
---


{/* cspell:ignore Eprint */}

# Transpile circuits remotely with the Qiskit Transpiler Service

<Admonition type="danger" title="Transpiler Service migration">
     As of 18 July 2025, the service is being migrated to support the new IBM Quantum&reg; Platform and is not available. For AI passes, you can use [local mode](/docs/guides/ai-transpiler-passes#run-the-ai-transpiler-passes-locally-or-on-the-cloud).

    The service is a beta release, subject to change.
    If you have feedback or want to contact the developer team, please use this [Qiskit Slack Workspace](https://qiskit.slack.com/archives/C06KF8YHUAU) channel.
</Admonition>


The Qiskit Transpiler Service provides transpilation capabilities on the cloud. In addition to the local Qiskit transpiler capabilities, your transpilation tasks can benefit from both IBM Quantum cloud resources and AI-powered transpiler passes.

The Qiskit Transpiler Service offers a Python library to seamlessly integrate this service and its capabilities into your current Qiskit patterns and workflows. This service is only available for IBM Quantum Premium Plan, Flex Plan, and On-Prem (via IBM Quantum Platform API) Plan users.


<span id="install-transpiler-service"></span>
## Install the qiskit-ibm-transpiler package

To use the Qiskit Transpiler Service, install the `qiskit-ibm-transpiler` package:

```sh
pip install qiskit-ibm-transpiler
```

The package automatically authenticates using your [IBM Quantum Platform credentials](/docs/guides/cloud-setup) aligned with how [Qiskit Runtime manages it](/docs/guides/initialize-account):
- Environment variable: `QISKIT_IBM_TOKEN`
- Configuration file `~/.qiskit/qiskit-ibm.json` (under the section `default-ibm-quantum`).

*Note*: This package requires Qiskit SDK v1.X.

## qiskit-ibm-transpiler transpile options

- `backend_name` (optional, str) - A backend name as it would be expected by QiskitRuntimeService (for example, `ibm_torino`). If this is set, the transpile method uses the layout from the specified backend for the transpilation operation. If any other option is set that impacts these settings, such as `coupling_map`, the `backend_name` settings are overridden.
- `coupling_map` (optional, List[List[int]]) - A valid coupling map list (for example, [[0,1],[1,2]]). If this is set, the transpile method uses this coupling map for the transpilation operation. If defined, it overrides any value specified for `target`.
- `optimization_level` (int) - The potential optimization level to apply during the transpilation process. Valid values are [1,2,3], where 1 is the least optimization (and fastest), and 3 the most optimization (and most time-intensive).
- `ai` ("true", "false", "auto") - Whether to use AI-powered capabilities during transpilation. The AI-powered capabilities available can be for `AIRouting` transpiling passes or other AI-powered synthesis methods. If this value is `"true"`, the service applies different AI-powered transpiling passes depending on the `optimization_level` requested. If  `"false"`, it uses the latest Qiskit transpiling features without AI. Finally, if `"auto"`, the service decides whether to apply the standard Qiskit heuristic passes or the AI-powered passes based on your circuit.
- `qiskit_transpile_options` (dict) - A Python dictionary object that can include any other option that is valid in the [Qiskit `transpile()` method](defaults-and-configuration-options). If the `qiskit_transpile_options` input includes `optimization_level`, it is discarded in favor of the `optimization_level` specified as parameter input. If the `qiskit_transpile_options` includes any option not recognized by the Qiskit `transpile()` method, the library raises an error.

For more information about the available `qiskit-ibm-transpiler` methods, see the [qiskit-ibm-transpiler API reference](/docs/api/qiskit-ibm-transpiler). To learn more about the service API, see the [Qiskit Transpiler Service REST API documentation.](/docs/api/qiskit-transpiler-service-rest)

## Examples

The following examples demonstrate how to transpile circuits using the Qiskit Transpiler Service with different parameters.

1. Create a circuit and call the Qiskit Transpiler Service to transpile the circuit with `ibm_torino` as the `backend_name`, 3 as the `optimization_level`, and without using AI during the transpilation.

In [ ]:
from qiskit.circuit.library import efficient_su2
from qiskit_ibm_transpiler.transpiler_service import TranspilerService

circuit = efficient_su2(101, entanglement="circular", reps=1)

cloud_transpiler_service = TranspilerService(
    backend_name="ibm_torino",
    ai="false",
    optimization_level=3,
)
transpiled_circuit = cloud_transpiler_service.run(circuit)

*Note*: you only can use backend_name devices you have access to with your IBM Quantum account. Apart from the `backend_name`, the `TranspilerService` also allows `coupling_map` as parameter.

2. Produce a similar circuit and transpile it, requesting AI transpiling capabilities by setting the flag `ai` to `True`:

In [ ]:
from qiskit.circuit.library import efficient_su2
from qiskit_ibm_transpiler.transpiler_service import TranspilerService

circuit = efficient_su2(101, entanglement="circular", reps=1)

cloud_transpiler_service = TranspilerService(
    backend_name="ibm_torino",
    ai="true",
    optimization_level=1,
)
transpiled_circuit = cloud_transpiler_service.run(circuit)

3. Produce a similar circuit and transpile it while letting the service to decide whether to use the AI-powered transpiling passes.

In [ ]:
from qiskit.circuit.library import efficient_su2
from qiskit_ibm_transpiler.transpiler_service import TranspilerService

circuit = efficient_su2(101, entanglement="circular", reps=1)

cloud_transpiler_service = TranspilerService(
    backend_name="ibm_torino",
    ai="auto",
    optimization_level=1,
)
transpiled_circuit = cloud_transpiler_service.run(circuit)

## Limits of the Qiskit Transpiler Service

Following are the most relevant limitations of the service:

- The maximum number of two-qubit gates per circuit in a transpilation job in any `ai` mode is 1 million.
- The maximum time allowed to run a transpilation process is 30 minutes per job.
- You must retrieve the transpilation result from the service within 20 minutes after the transpilation process ends. After 20 minutes, the job result is discarded.
- The maximum time a set of circuits can live in the internal queue while waiting to be transpiled is 120 minutes. After that time, if the job has not been transpiled, it is discarded.
- The maximum number of qubits has not been determined. The service has been tested on 900+ qubits.

## Citation

If you use any AI-powered feature from the Qiskit Transpiler Service in your research, use the following recommended citation:

```
@misc{2405.13196,
Author = {David Kremer and Victor Villar and Hanhee Paik and Ivan Duran and Ismael Faro and Juan Cruz-Benito},
Title = {Practical and efficient quantum circuit synthesis and transpiling with Reinforcement Learning},
Year = {2024},
Eprint = {arXiv:2405.13196},
}
```

## Next steps

<Admonition type="tip" title="Recommendations">
    - Learn how to create [AI transpiler passes.](ai-transpiler-passes)
    - Learn [how to transpile circuits](/docs/guides/circuit-transpilation-settings) as part of the Qiskit patterns workflow using Qiskit Runtime.
    - Review the [Qiskit Transpiler Service Python client](/docs/api/qiskit-ibm-transpiler) documentation.
</Admonition>